In [ ]:
import datetime
import time
import redis
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [ ]:
%matplotlib nbagg
HOSTNAME = "devstack02"
r = redis.StrictRedis(host=HOSTNAME,port=6379,db=0)


In [ ]:
unix_now = int(time.mktime(datetime.datetime.now().timetuple()))

def get_keys(conn):
    return conn.keys()

def get_range(conn,key,start,end):
    return conn.zrange(key,start,end)

def get_rangebyscore(conn,key,start,end):
    return conn.zrangebyscore(key,start,end)

def load(time, num):
    list_array = np.empty((0, num))
    all_keys = get_keys(r)
    for key in all_keys:
        if HOSTNAME in key:
            if 'cpu' in key:
                key_array = []
                n = np.array(get_rangebyscore(r, key, time - num, time))

                for i,each_array in enumerate(n):
                    each_array = np.array(each_array.split(":"))
                    each_array = np.delete(each_array, 0)
                    each_array = each_array.astype(float)
                    if each_array.size >1:
                        if i == 0:
                            tmp_array = np.empty((0,each_array.size))
                        tmp_array = np.vstack((tmp_array,each_array))
                    else:
                        key_array = np.append(key_array,each_array)
                if each_array.size > 1:
                    list_array = np.vstack((list_array,tmp_array.T))
                else:
                    list_array = np.vstack((list_array,key_array))
                    
    return list_array.T

def diff_data(time, num):
    list_array = np.empty((0, num))
    all_keys = get_keys(r)
    for key in all_keys:
        if HOSTNAME in key:
                   
            if 'cpu' in key:
                key_array = []
                n = np.array(get_rangebyscore(r, key, time - num, time))

                for i,each_array in enumerate(n):
                    each_array = np.array(each_array.split(":"))
                    each_array = np.delete(each_array, 0)
                    each_array = each_array.astype(float)
                    if each_array.size >1:
                        if i == 0:
                            tmp_array = np.empty((0,each_array.size))
                        tmp_array = np.vstack((tmp_array,each_array))
                    else:
                        key_array = np.append(key_array,each_array)
                if each_array.size > 1:
                    list_array = np.vstack((list_array,tmp_array.T))
                else:
                    list_array = np.vstack((list_array,key_array))
    list_array = np.diff(list_array)
    
    return list_array.T


def pca_fit(pca_list):
    load_data_nparray = np.array(pca_list)
    pca = PCA(n_components=2)
    pca.fit(load_data_nparray)
    return pca

def pca_transform(fit, pca_list):
    transformed = fit.transform(pca_list)
    #print transformed
    return transformed

def kmeans_fit(data):
    fit = KMeans(n_clusters=3).fit(data)
    return fit

def kmeans_predict(fit, data):
    prediction = fit.predict(data)
    return prediction

def predict_now(fit, data):
    now = fit.predict(data)
    return now

data = diff_data(unix_now,7200)
now_data = diff_data(unix_now,5)

pca_fit = pca_fit(data)

data = pca_transform(pca_fit, data)
#print data
now_data = pca_transform(pca_fit,now_data)
#print now_data
fit = kmeans_fit(data)
cluster = kmeans_predict(fit,data)
#print fit.cluster_centers_[0]
graph_size=1000
plt.xticks(np.arange(-graph_size,graph_size,100))
plt.yticks(np.arange(-graph_size,graph_size,100))
plt.scatter(data[:,0],data[:,1], c=cluster)
plt.scatter(now_data[:,0],now_data[:,1],color='red')
print unix_now
plt.show()

In [ ]:
fig = plt.figure()

graph_size=1000
plt.xticks(np.arange(-graph_size,graph_size,100))
plt.yticks(np.arange(-graph_size,graph_size,100))

#ims = []
def update(i, fig, im):
    unix_now = int(time.mktime(datetime.datetime.now().timetuple()))
    now_data = diff_data(unix_now,6)
    now_data = pca_transform(pca_fit,now_data)

    if len(im) > 0:
        im[0].remove()
        im.pop()
        
    im.append(plt.scatter(now_data[:,0],now_data[:,1],color='red', marker="o"))

im=[]
plt.title("K-means Cluster")
plt.scatter(data[:,0],data[:,1], c=cluster, marker="o", alpha=0.1)
ani = animation.FuncAnimation(fig, update, fargs = (fig,im), interval=1000)
#ani.save('/tmp/k-means.mp4')
plt.show()